In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Refeence format
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models

# alpaca-cleaned

In [2]:
# Expected dataset have openai-like messages format

dataset = load_dataset('yahma/alpaca-cleaned', split='train')

one_row = dataset[0]
def get_messages(one_row):
    # return openai messages format
    # Expected is a list of dictionaries with the keys "role" and "content", roles are "user" and "assistant" or system, note that gemma models does not support system role
    messages = [
        {"role": "user", "content": one_row["instruction"] + one_row["input"]},
        {"role": "assistant", "content": one_row["output"]},
    ]
    
    return {"messages": messages}

# Apply the chat template to each example in the dataset
dataset = dataset.map(get_messages)
dataset.save_to_disk('../data/alpaca-cleaned')
dataset

Saving the dataset (0/1 shards):   0%|          | 0/51760 [00:00<?, ? examples/s]

Dataset({
    features: ['output', 'input', 'instruction', 'messages'],
    num_rows: 51760
})

# OpenO1-SFT

{'instruction': "Consider a regular octagon. How many different triangles can be formed if the octagon is placed inside a circle and we can also use the center of the circle as a vertex for the triangles? Let's think step by step.", 'output': "<Thought>\nAlright, I need to figure out how many different triangles can be formed in a regular octagon that's placed inside a circle, with the added twist that the center of the circle can also be used as a vertex for the triangles. Let's break this down step by step.\n\nFirst, let's recall that a regular octagon has 8 equal sides and 8 equal angles. Since it's placed inside a circle, each vertex of the octagon lies on the circumference of the circle. Additionally, we have the center of the circle as an extra point that can be used as a vertex for forming triangles.\n\nSo, in total, we have 9 points to consider: the 8 vertices of the octagon and the center of the circle.\n\nNow, to form a triangle, we need to select any 3 of these 9 points. The

In [3]:
dataset = "O1-OPEN/OpenO1-SFT"
# Expected dataset have openai-like messages format

dataset = load_dataset(dataset, split='train')
one_row = dataset[0]
print(one_row)


def get_messages(one_row):
    # return openai messages format
    # Expected is a list of dictionaries with the keys "role" and "content", roles are "user" and "assistant" or system, note that gemma models does not support system role
    messages = [
        {"role": "user", "content": one_row["instruction"]},
        {"role": "assistant", "content": one_row["output"]},
    ]
    
    return {"messages": messages}

# Apply the chat template to each example in the dataset
dataset = dataset.map(get_messages)
dataset = dataset.shuffle(42).select(range(6000),)
dataset.select(range(0, 5000, 1)).save_to_disk('../data/OpenO1-SFT-5k')
dataset.select(range(5000, 6000, 1)).save_to_disk('../data/OpenO1-SFT-1k')
dataset

{'instruction': "Consider a regular octagon. How many different triangles can be formed if the octagon is placed inside a circle and we can also use the center of the circle as a vertex for the triangles? Let's think step by step.", 'output': "<Thought>\nAlright, I need to figure out how many different triangles can be formed in a regular octagon that's placed inside a circle, with the added twist that the center of the circle can also be used as a vertex for the triangles. Let's break this down step by step.\n\nFirst, let's recall that a regular octagon has 8 equal sides and 8 equal angles. Since it's placed inside a circle, each vertex of the octagon lies on the circumference of the circle. Additionally, we have the center of the circle as an extra point that can be used as a vertex for forming triangles.\n\nSo, in total, we have 9 points to consider: the 8 vertices of the octagon and the center of the circle.\n\nNow, to form a triangle, we need to select any 3 of these 9 points. The

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'messages'],
    num_rows: 6000
})

In [14]:
dataset[0]

{'instruction': 'Anita, Indu and Geeta can do a piece of work in 18 days, 27 days and 36 days respectively. They start working together. After working for 4 days, Anita goes away and Indu leaves 7 days before the work is finished. Only Geeta remains at work from beginning till end. In how many days was the whole work done?',
 'output': "<Thought>\nAlright, I need to figure out how long it took for Anita, Indu, and Geeta to complete the work under the given conditions. Let's break down the information step by step. \n\nFirst, Anita can complete the work in 18 days, Indu in 27 days, and Geeta in 36 days. So their individual work rates are:\n- Anita: 1/18 work per day\n- Indu: 1/27 work per day\n- Geeta: 1/36 work per day\n\nThey start working together, which means their combined work rate initially is 1/18 + 1/27 + 1/36. I need to calculate this sum. Let's find a common denominator for 18, 27, and 36. The least common multiple of these numbers is 108. \n\nSo,\n1/18 = 6/108\n1/27 = 4/108\